In [1]:
# @title libraries
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import warnings
import matplotlib.pyplot as plt
#from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import plotly.express as px
from sklearn.linear_model import Lasso
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE
import pickle
import csv
import requests




from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso#
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
def load_file(github_url):
  # Fetch the file from GitHub
  response = requests.get(github_url)
  if response.status_code == 200:
      # Load the pickle file
      obj = pickle.loads(response.content)
      print("Pickle file loaded successfully!")
      return obj
  else:
      print("Failed to fetch the pickle file. Status code:", response.status_code)

In [ ]:
model_path = 'https://raw.githubusercontent.com/ArvinCorotana/ML/main/best_reg_model.pkl'
imputer_path = 'https://raw.githubusercontent.com/ArvinCorotana/ML/main/reg_imputer.pkl'
scaler_path = 'https://raw.githubusercontent.com/ArvinCorotana/ML/main/reg_scaler.pkl'
top_features_path = 'https://raw.githubusercontent.com/ArvinCorotana/ML/main/reg_top_features.pkl'

loaded_model = load_file(model_path)
imputer = load_file(imputer_path)
scaler = load_file(scaler_path)
top_features = load_file(top_features_path)

Pickle file loaded successfully!
Pickle file loaded successfully!
Pickle file loaded successfully!
Pickle file loaded successfully!


In [ ]:
class Preprocess:
  def __init__(self, data, label=None, columns_to_keep=None, scaler=None, imputer=None, test=True):
    self.data = data
    self.label = label
    self.categorical_features = None
    self.numerical_features = None
    self.test = test
    self.columns_to_keep = columns_to_keep
    self.scaler = scaler
    self.imputer = imputer

  def preprocess_data(self):
    dataset = self.data
    # ids = dataset[['ID']]
    # dataset = dataset.drop(['ID'], axis=1)
    self.categorical_features = dataset.columns[1:10]
    self.numerical_features = [i for i in list(dataset.columns) if not i in self.categorical_features]


    # # Handle missing values
    dataset.replace(999, np.nan, inplace=True)
    # print('r', dataset.isna().sum().sum())
    dataset = self.imputation(dataset)
    # print('mv', dataset.isna().sum().sum())

    # Handle outliers
    dataset = self.handle_outliers(dataset)
    # print('o', dataset.isna().sum().sum())

    # Feature Selection
    if self.test:
      dataset = dataset[self.columns_to_keep]
    else:
      dataset = self.feature_selection(dataset, self.label, num_features_to_select=30)
    # print('fs', dataset.isna().sum().sum())
    # Correct skewness
    # dataset = self.correct_skewness(dataset)

    # Normalisation
    dataset, self.label = self.normalize(dataset, self.label)
    # print('n', dataset.isna().sum().sum())

    # Split data
    # print(dataset.shape, self.label.shape)
    if not self.test:
      X_train, X_test, y_train, y_test = self.split_train_test(dataset, self.label)
      return X_train, X_test, y_train, y_test
    else:
      return dataset



  def imputation(self, dataset):
    columns=dataset.columns
    index=dataset.index
    imputer = self.imputer if self.test else SimpleImputer(strategy='most_frequent')
    dataset = self.imputer.transform(dataset) if self.test else imputer.fit_transform(dataset)
    dataset = pd.DataFrame(dataset, columns=columns, index=index)
    return dataset

  def handle_outliers(self, data, strategy='IQR'):
    # print('removing outliers')
    data_copy = data.copy()
    Q1 = data_copy.quantile(0.25, numeric_only=True)
    Q3 = data_copy.quantile(0.75, numeric_only=True)
    # print(Q3)
    IQR = Q3 - Q1
    lower_lim = Q1 - (1.5 * IQR)
    upper_lim = Q3 + (1.5 * IQR)

    for column in data_copy:
        outliers = ((data_copy[column] < lower_lim[column]) |
                    (data_copy[column] > upper_lim[column]))

        # print(data_copy)
        data_copy.loc[outliers, column] = data_copy.loc[outliers, column].apply(
            lambda x: lower_lim[column] if x < lower_lim[column] else upper_lim[column]
        )

    return data_copy

  def feature_selection(self, X,y, num_features_to_select=50):
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      selected_features = self.columns_to_keep if self.test else select_features_mrmr(X, np.array(y), num_features_to_select=num_features_to_select)
      # print(selected_features)

      X = X[selected_features]
      return X

  def correct_skewness(self, X):
    skewed_features = X.apply(lambda x: x.skew()).sort_values(ascending=False)
    # print(skewed_features)

    # Identify features with skewness greater than a threshold (e.g., 0.5)
    skewness_threshold = 0.1
    skewed_features = skewed_features[abs(skewed_features) > skewness_threshold]

    # Apply log transformation to correct skewness
    for feature in skewed_features.index:
        X[feature] = np.log1p(X[feature])

    return X

  def normalize(self, X,y):
    columns = X.columns
    scaler = self.scaler if self.test else StandardScaler()
    X = scaler.transform(X) if self.test else scaler.fit_transform(X)
    X = pd.DataFrame(X, columns=columns)

    # scaler = StandardScaler()
    # y = scaler.fit_transform(np.array(y).reshape(-1,1))
    # y = pd.DataFrame(y)

    return X,y

  def split_train_test(self,X,y,test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=test_size)
    print(f'The design matrix shape = {X_train.shape}')
    print(f'The target matrix shape = {y_train.shape}')
    return X_train, X_test, y_train, y_test

def load_data(filepath):
    dataset = pd.read_excel(filepath, sheet_name='Sheet1')
    # dataset = dataset.drop(['pCR (outcome)'], axis=1)
    return dataset


In [ ]:
url_test = 'https://raw.githubusercontent.com/ArvinCorotana/ML/main/TestDatasetExample1.xls'
test_data = pd.read_excel(url_test, sheet_name='Sheet1')

#Add your file here
#test_data = 

test_ids = test_data[['ID']]

test_data = test_data.drop(['ID'], axis=1)
test_data.head()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,56.881588,0,0,0,1,3,3,999,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
1,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553
2,58.234086,0,0,0,1,3,3,1,1,4,...,0.328377,0.328377,3.785966,0.003185,3.607821e+06,223.279556,0.001334,0.101628,0.010844,0.001194


In [ ]:
test_data.isna().sum().sum()

0

In [ ]:
test_data[['original_firstorder_Minimum']]

,original_firstorder_Minimum
0,-1.120653
1,-1.035116
2,-1.623018


In [ ]:
test_data.shape

(3, 117)

In [ ]:
X_test = Preprocess(test_data, columns_to_keep=top_features, imputer=imputer, scaler=scaler, test=True).preprocess_data()
X_test.head()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_gldm_SmallDependenceEmphasis,original_glrlm_RunLengthNonUniformity,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_LargeAreaEmphasis,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness
0,0.598583,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.333333,...,0.438429,0.332924,0.606219,0.052225,0.497122,0.202694,0.240724,0.196790,0.358746,0.066399
1,0.653675,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,0.666667,...,0.695502,0.105923,0.705563,0.121733,0.366253,0.005818,0.310981,0.360914,0.065613,0.351117
2,0.622477,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.000000,...,0.462715,0.569977,0.633270,0.319464,0.883646,0.156700,0.139447,0.387054,0.611248,0.034541


In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=['RelapseFreeSurvival (outcome)'])
y_pred

,RelapseFreeSurvival (outcome)
0,59.720000
1,44.751667
2,46.447381


In [2]:
output = pd.concat([test_ids, y_pred], axis=1)

output.to_csv('Test_RFS.csv',index=False)


NameError: name 'test_ids' is not defined